In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

import datetime

In [ ]:
nc1 = pd.read_csv('get_shortfall_sorted_firstTime.csv')
nc1 = nc1.drop(nc1[nc1['Data'] == '#N/A Limit: Monthly'].index)
nc1 = nc1.dropna()
nc1['Year'] = nc1['NAME'].apply(lambda x: x[x.find('-')-4:x.find('-'):])

In [ ]:
def make_dict(x):
    d = {}
    for n in range(0,len(x)-1,2):
        if (x[n+1] == '#N/A N/A'): x[n+1]= np.nan
        else : x[n+1] = float(x[n+1])
        date = str(x[n]).split('/')
        d[datetime.date(int(date[2]),int(date[0]),int(date[1]))] = x[n+1]
    return d
make_dict_vectorize = np.vectorize(make_dict)

In [ ]:
nc1['Data_dict'] = make_dict_vectorize(nc1['Data'].str.split(','))

In [ ]:
#plt.plot(list(nc1['Data_dict'][1].keys()),list(nc1['Data_dict'][1].values()))

In [ ]:
def make_dict_noNaN(x):
    d = {}
    for n in range(0,len(x)-1,2): 
        if (x[n+1] == '#N/A N/A'): pass
        else :
            x[n+1] = float(x[n+1])
            date = str(x[n]).split('/')
            d[datetime.date(int(date[2]),int(date[0]),int(date[1]))] = x[n+1]
    if (d):return d
    else : return np.nan
make_dict_noNaN_vectorize = np.vectorize(make_dict_noNaN)

In [ ]:
nc1['Data_dict_noNaN'] = make_dict_noNaN_vectorize(nc1['Data'].str.split(','))

In [ ]:
nc1 = nc1.dropna()
nc1['Data_dict_plot'] = nc1['Data_dict_noNaN'].apply(lambda x: (list(x.keys()),list(x.values())))

In [ ]:
nc1['Year'].unique()

In [ ]:
def earliest(x):
    return x[0][-1]
earliest_vectorize = np.vectorize(earliest)
early = earliest_vectorize(nc1['Data_dict_plot'])
print(early)

In [ ]:
nc1.index = np.arange(1,nc1.shape[0]+1)
nc1 = nc1.dropna()

In [ ]:
from pathlib import Path
import os

# Reset index and drop nan
nc1.index = np.arange(1,nc1.shape[0]+1)
nc1 = nc1.dropna()

# year parameter
year_from = "2006"
year_end = "2012"

# directory parameter
data_folder = Path(r'C:\Users\cchen07\Desktop\NC1_images') 
#sub_folder = data_folder / (year_from+"to"+year_end)

#data_folder = Path("D:/resMBS/NC1_images")
suffix = ''
if suffix : sub_folder = data_folder / (year_from+"to"+year_end+"_"+suffix)
else : sub_folder = data_folder / (year_from+"to"+year_end)

# create directory
try:
    os.mkdir(sub_folder)
except OSError:  
    print ("Creation of the directory %s failed" % sub_folder)
else:  
    print ("Successfully created the directory %s " % sub_folder)

#for n in range(1,10):
for n in range(0,nc1.shape[0]):
    nc1_n = nc1.iloc[n]
    nc1_n_Data_dict_plot = nc1_n['Data_dict_plot']
    fig = plt.figure()
    plt.xlim(np.datetime64(year_from), np.datetime64(year_end))
    
    #plt.plot(nc1_n_Data_dict_plot[0],nc1_n_Data_dict_plot[1],marker='o')
    
    values = pd.Series(nc1_n_Data_dict_plot[1]).iloc[::-1]
    cumsum = values.cumsum()
    dates = pd.Series(nc1_n_Data_dict_plot[0]).iloc[::-1]
    plt.plot(dates,values,'o',dates,cumsum,'^')
    
    plt.xlabel('Time')
    plt.ylabel('Short Fall Amount')
    name = str(nc1_n['NAME'])+' - '+ str(nc1_n['SHORTFALL_TYPE'])
    plt.title(name)
    file_name = sub_folder / (name+".png")
    plt.savefig(file_name,dpi=300,orientation='landscape',format='png')
    plt.close(fig)

In [ ]:
from pathlib import Path
import os

# Reset index and drop nan
nc1.index = np.arange(1,nc1.shape[0]+1)
nc1 = nc1.dropna()

# year parameter
year_from = "2006"
year_end = "2012"

# directory parameter
data_folder = Path(r'C:\Users\cchen07\Desktop\NC1_images') 
#sub_folder = data_folder / (year_from+"to"+year_end)

#data_folder = Path("D:/resMBS/NC1_images")
suffix = 'standardized'
if suffix : sub_folder = data_folder / (year_from+"to"+year_end+"_"+suffix)
else : sub_folder = data_folder / (year_from+"to"+year_end)

# create directory
try:
    os.mkdir(sub_folder)
except OSError:  
    print ("Creation of the directory %s failed" % sub_folder)
else:  
    print ("Successfully created the directory %s " % sub_folder)

#for n in range(1,10):
for n in range(0,nc1.shape[0]):
    nc1_n = nc1.iloc[n]
    nc1_n_Data_dict_plot = nc1_n['Data_dict_plot']
    fig = plt.figure()
    plt.xlim(np.datetime64(year_from), np.datetime64(year_end))
    
    #plt.plot(nc1_n_Data_dict_plot[0],nc1_n_Data_dict_plot[1],marker='o')
    
    values = pd.Series(nc1_n_Data_dict_plot[1]).iloc[::-1]
    standardized_values = (values-values.mean()) / values.std()
    standardized_cumsum = standardized_values.cumsum()
    dates = pd.Series(nc1_n_Data_dict_plot[0]).iloc[::-1]
    plt.plot(dates,standardized_values,'o',dates,standardized_cumsum,'^')
    
    plt.xlabel('Time')
    plt.ylabel('Short Fall Amount')
    name = str(nc1_n['NAME'])+' - '+ str(nc1_n['SHORTFALL_TYPE'])
    plt.title(name)
    file_name = sub_folder / (name+".png")
    plt.savefig(file_name,dpi=300,orientation='landscape',format='png')
    plt.close(fig)

In [ ]:
#a.apply(lambda x: plt.plot)

In [ ]:
""""
def export_pic(x):
    plt.plot(x)
    return plt.savefig('image/a.png')
export_pic_vectorize = np.vectorize(export_pic)

export_pic_vectorize(nc1['Data_dict_plot'])
""""